In [184]:
from concorde import Problem, run_concorde
import tsplib95
import pandas as pd
import json

ImportError: cannot import name 'Problem' from 'concorde' (/home/gui/.local/lib/python3.10/site-packages/concorde/__init__.py)

In [ ]:
with open('parameters.json') as f:
    parameters = json.load(f)

In [ ]:
with open('playlist.json') as f:
    data = json.load(f)

data

[{'id': '0NWPxcsf5vdjdiFUI8NgkP',
  'name': 'Hey Joe',
  'album': 'Are You Experienced',
  'artist': 'Jimi Hendrix',
  'genres': ['acid rock',
   'album rock',
   'classic rock',
   'hard rock',
   'proto-metal',
   'psychedelic rock',
   'rock'],
  'related_artists': ['Cream',
   'Canned Heat',
   'The Yardbirds',
   'Derek & The Dominos',
   'Steppenwolf',
   'Janis Joplin',
   'Ten Years After',
   'Free',
   'Jefferson Airplane',
   'The Doors',
   'The Animals',
   'Big Brother & The Holding Company',
   'Stevie Ray Vaughan',
   'Rory Gallagher',
   'Jim Morrison',
   'Traffic',
   'John Mayall & The Bluesbreakers',
   'Allman Brothers Band',
   'Albert King',
   'Freddie King'],
  'related_genres': ['mellow gold',
   'soft rock',
   'singer-songwriter',
   'art rock',
   'instrumental rock',
   'blues rock',
   'roots rock',
   'british invasion',
   'traditional blues',
   'electric blues',
   'memphis soul',
   'folk',
   'progressive rock',
   'folk rock',
   'texas blues',
  

In [ ]:
def compare_artist_album_similarity(data_a:dict, data_b:dict) -> int:
    distance = 0

    if (data_a["artist"] == data_b["artist"]):
        distance += parameters["same_artist_bonus"]
        
        if (data_a["album"] == data_b["album"]):
            distance += parameters["same_album_bonus"]

    return distance


def compare_intersections(a:list, b:list, multiplier:list) -> int:
    if a and b:
        set_a = set(a)
        set_b = set(b)

        intersection = set_a.intersection(set_b)

        intersection_size = len(intersection)

        if intersection_size > 0:
        
            total_size = len(set_a) + len(set_b)

            CONSTANT = 100

            similarity = (intersection_size / total_size) * CONSTANT

            return similarity *multiplier

    return 0

In [ ]:
def initialize_adjacency_matrix(size:int) -> list:
    return [[0 for col in range(size)] for row in range(size)]


def populate_adjacency_matrix(adjacency_matrix:list, size:int) -> list:
    for i in range(size):
        for j in range(size):
            if i != j:
                distance = parameters['initial_distance']

                distance -= compare_artist_album_similarity(data[i], data[j])

                distance -= compare_intersections(
                    data[i]["genres"], data[j]["genres"], parameters["genres_intersection_multiplier"])
                distance -= compare_intersections(
                    data[i]["related_genres"], data[j]["related_genres"], parameters["related_genres_intersection_multiplier"])
                distance -= compare_intersections(
                    data[i]["related_artists"], data[j]["related_artists"], parameters["related_artists_intersection_multiplier"])
                distance -= compare_intersections(
                    data[i]["generic_genres"], data[j]["generic_genres"], parameters["generic_genres_intersection_multiplier"])

                adjacency_matrix[i][j] = int(distance);
    
    return adjacency_matrix


def print_adjacency_matrix(adjacency_matrix:list, size:int) -> None:
    for i in range(size):
        for j in range(size):
            print("{:5d}".format(adjacency_matrix[i][j]), end=" ")
        print()    

def create_tsplib_file(adjacency_matrix, filename):
  with open(filename, 'w') as f:
    f.write('NAME: tsp_problem\n')
    f.write('COMMENT: This is a TSP problem.\n')
    f.write('DIMENSION: {}\n'.format(len(adjacency_matrix)))
    f.write('EDGE_WEIGHT_TYPE: EXPLICIT\n')
    f.write('EDGES:\n')

    for i in range(len(adjacency_matrix)):
      for j in range(i + 1, len(adjacency_matrix)):
        f.write('{} {} {}\n'.format(i, j, adjacency_matrix[i][j]))

        

In [ ]:
SIZE = len(data)
adjacency_matrix = initialize_adjacency_matrix(SIZE)
adjacency_matrix = populate_adjacency_matrix(adjacency_matrix, SIZE)
create_tsplib_file(adjacency_matrix, "test.tsp")
print_adjacency_matrix(adjacency_matrix, SIZE)

    0   879   986   922   935   853   879   953  1000 
  879     0   986   985   926   781   290   942   984 
  986   986     0   984   955   973   986   928   866 
  922   985   984     0   953   959   985   985  1000 
  935   926   955   953     0   483   926   917   981 
  853   781   973   959   483     0   781   902   969 
  879   290   986   985   926   781     0   942   984 
  953   942   928   985   917   902   942     0   984 
 1000   984   866  1000   981   969   984   984     0 


In [ ]:
problem1 = Problem.from_matrix(adjacency_matrix)
solution1 = run_concorde(problem1)


Problem Name: tsp_problem
This is a TSP problem.
Number of Nodes: 9
Explicit Lengths (CC_MATRIXNORM)


ERROR: Didn't find the data


RuntimeError: Error in loading test.tsp